In [1]:
import pathlib
import zipfile
import os
import tensorflow as tf
from PIL import Image as Image
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import random
import matplotlib.pyplot as plt
from keras import regularizers
from sklearn.model_selection import train_test_split

In [2]:
clean_dataset = "../DatasetL2/Project_Dataset_Unzip"

data_dir = clean_dataset

data_dir = pathlib.Path(data_dir)

In [3]:
image_h = 75
image_w = 75
batch_s = 32

In [4]:
def add_noise(img):
    noise_factor = 0.9
    noise = noise_factor * np.random.normal(loc=0.0, scale=1.0, size=img.shape)
    img = img + noise
    np.clip(img, 0., 1.)
    return img

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=True,
        validation_split=0.20,
        #preprocessing_function=add_noise,
        )

train_generator = train_datagen.flow_from_directory(
        clean_dataset,
        subset = 'training',
        target_size=(image_h, image_w),
        batch_size=32,
        class_mode=None,
        )

test_generator = train_datagen.flow_from_directory(
        clean_dataset,
        subset = 'validation',
        target_size=(image_h, image_w),
        batch_size=32,
        class_mode=None)

Found 120 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


In [12]:
print(test_generator.__len__())

1


In [5]:
import tensorflow as tf

input_img = keras.Input(shape=(image_h * image_w,))
encoded = layers.Dense(2048, activation='relu')(input_img)
encoded = layers.Dense(512, activation='relu')(encoded)
encoded = layers.Dense(256, activation='relu')(encoded)
encoded = layers.Dense(128, activation='relu')(encoded)
encoded = layers.Dense(64, activation='relu')(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(256, activation='relu')(decoded)
decoded = layers.Dense(512, activation='relu')(decoded)
decoded = layers.Dense(2048, activation='relu')(decoded)
decoded = layers.Dense(5625, activation='sigmoid')(decoded)

auto_encoder = keras.Model(input_img, decoded)
auto_encoder.compile(optimizer = 'adam',
                  loss = 'mean_absolute_error')
auto_encoder.fit(train_generator,
                epochs=10,
                batch_size=batch_s,
                shuffle=True,
                validation_data=test_generator,
                )
auto_encoder.summary()

Epoch 1/10


ValueError: in user code:

    C:\Users\nicos\PycharmProjects\DataScience\venv\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\nicos\PycharmProjects\DataScience\venv\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\nicos\PycharmProjects\DataScience\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\nicos\PycharmProjects\DataScience\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\nicos\PycharmProjects\DataScience\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\nicos\PycharmPro

In [ ]:
tf.keras.utils.plot_model(auto_encoder, show_shapes=True)

In [ ]:
decoded_imgs = auto_encoder.predict(test_generator, verbose=1)
print(decoded_imgs.shape)

In [ ]:
print(decoded_imgs[1].shape)

In [ ]:
def display_image(X, n):
    plt.figure(figsize=(20, 2))
    for i in range(n):
        ax = plt.subplot(1,n,i+1)
        plt.imshow(X[i].reshape(image_h, image_w))
        #plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
display_image(decoded_imgs,10)